In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import scipy 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import ast
from scipy.interpolate import make_interp_spline
from sklearn import metrics
import seaborn as sns
from scipy.fft import fft,ifft
import warnings
warnings.filterwarnings("ignore")

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Deep_learning/airline-price-prediction(raw_classification).csv')
#dataset = pd.read_csv('airline-price-prediction.csv')
dataset.head()

In [ ]:
np.sum(dataset.duplicated())

In [ ]:
dataset.dtypes

In [ ]:
np.sum(dataset.isna())